# Make your own custom two body model

In this example, a user can follow the instructions in /feasst/plugin/example/include/model_example.h to make their own custom model.

In [1]:
import sys
import subprocess
import unittest
import pandas as pd

Write a two site xyz file for testing the energy with a cubic box length of 8, one particle on the origin and the second particle a distance of 1.125 away from the origin along the z-axis.

In [2]:
with open('example_two.xyz', 'w') as myfile: myfile.write("""2
-1 8 8 8
0 0 0 0
0 0 0 1.125
""")

Prepare to run a feasst simulation with the given parameters

In [3]:
def run_fst(params):
    with open("launch.txt", "w") as myfile: myfile.write("""
MonteCarlo
Configuration particle_type0 /feasst/plugin/example/particle/jagla.fstprt xyz_file example_two.xyz
Potential Model ModelExample num_discretized_steps {num_discretized_steps}
ThermoParams beta 1
Metropolis
Log output_file log.csv max_precision true clear_file true
Run num_trials 1
""".format(**params))
    syscode = subprocess.call("../../../build/bin/fst < launch.txt > launch.log", shell=True, executable='/bin/bash')
    if syscode > 0: sys.exit(1)

Run the test and check the energy.

In [4]:
class TestExampleText(unittest.TestCase):
    def test_model(self):
        run_fst({"num_discretized_steps": "0"})
        df = pd.read_csv('log.csv')
        self.assertEqual(2, df['p0'][0])
        self.assertEqual(0.5, df['energy'][0])
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_model (__main__.TestExampleText) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


Optionally test the python interface (if installed).

In [ ]:
import feasst

class TestExamplePython(unittest.TestCase):
    def test_model(self):
        example = feasst.MakeModelExample({"num_discretized_steps": "0"})
        self.assertEqual(0, example.energy(3.**2, 0, 0, feasst.ModelParams()))
        self.assertEqual(0, example.num_discretized_steps())

unittest.main(argv=[''], verbosity=2, exit=False)

test_model (__main__.TestExamplePython) ... 

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!